In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os

# System independent file management

In [3]:
from os.path import expanduser, join

Get the user directlry and locate the data relative to it

In [4]:
home_dir = expanduser('~')
data_dir = join(home_dir,'simdb','data','shear_zone','example_01')
data_dir

'/home/rch/simdb/data/shear_zone/example_01'

Get the list of files within the directory and sort them alphabetically

In [5]:
results = [join(data_dir,each) 
           for each in sorted(os.listdir(data_dir)) 
           if each.endswith('.csv')]
results

['/home/rch/simdb/data/shear_zone/example_01/Load_000_KN.csv',
 '/home/rch/simdb/data/shear_zone/example_01/Load_050_KN.csv',
 '/home/rch/simdb/data/shear_zone/example_01/Load_103_KN.csv',
 '/home/rch/simdb/data/shear_zone/example_01/Load_150_KN.csv',
 '/home/rch/simdb/data/shear_zone/example_01/Load_191_KN.csv']

Verify - read a single file

In [6]:
np.loadtxt(results[4], dtype=float, 
            skiprows=1, delimiter=',', usecols=(4,7), unpack=False)

array([[ 1.651,  1.651],
       [-5.978, -5.978],
       [ 1.653,  1.653],
       ...,
       [-5.866, -5.866],
       [-0.801, -0.801],
       [-9.623, -9.623]])

# Read the displacements into a single two dimensional array

Read all files as array using `loadtxt` and put them into a list, than make an array from the list, show the shape. Denote the dimensionality of the array using the index names, i.e. `u_te`, with `t` denoting the time index and `e` data entry index 

In [9]:
u_te = np.array([
    np.loadtxt(csv_file, dtype=float,  
               skiprows=1, delimiter=',', usecols=(4), unpack=False)
    for csv_file in results
], dtype=np.float_)
u_te.shape
u_te = np.hstack([u_te])

Reshape the array such that the last two indexes are the x and y displacements, the array indexes are then
`tpa`
where `p` denotes the point index and `a` the dimensional index

In [10]:
n_t, n_e = u_te.shape # get the dimensions of the time and entry dimensions
u_tpa = u_te.reshape(n_t,-1,2)
u_tpa.shape

(5, 863, 2)

In [11]:
u_tpa[-1,:34,1]

array([-5.978, -6.09 , -6.204, -6.325, -6.493, -6.668, -6.796, -6.927,
       -7.059, -7.217, -7.388, -7.549, -7.697, -7.805, -7.918, -8.033,
       -8.14 , -8.244, -8.345, -8.449, -8.702, -8.766, -8.835, -8.913,
       -8.982, -9.057, -9.127, -9.199, -9.268, -9.381, -9.468, -9.549,
       -9.617, -9.679])

In [12]:
add_point_tpa=np.zeros((5,1,2))
uu_tpa = np.concatenate([u_tpa, add_point_tpa],axis=1)
uu_tpa.shape

(5, 864, 2)

In [13]:
n_cols, n_rows = 36, 24

In [14]:
u_tjia = uu_tpa.reshape(n_t,n_rows, n_cols,2)
u_tija = np.einsum('tjia->tija', u_tjia)
u_tjia.shape, u_tija.shape

((5, 24, 36, 2), (5, 36, 24, 2))

In [15]:
u1_tija = u_tija[:,:-1,:,:]

In [16]:
u1_tija[-1,:,0,1]

array([-5.978, -6.09 , -6.204, -6.325, -6.493, -6.668, -6.796, -6.927,
       -7.059, -7.217, -7.388, -7.549, -7.697, -7.805, -7.918, -8.033,
       -8.14 , -8.244, -8.345, -8.449, -8.702, -8.766, -8.835, -8.913,
       -8.982, -9.057, -9.127, -9.199, -9.268, -9.381, -9.468, -9.549,
       -9.617, -9.679, -9.733])

# Generate the grid of initial positions

In [17]:
d_x, d_y = 14, 14

In [18]:
x_range = np.arange(n_cols-1)[::-1] * d_x
y_range = np.arange(n_rows) * d_y
y_ij, x_ij = np.meshgrid(y_range, x_range)

In [19]:
X_aij = np.array([x_ij, y_ij])
X_ija = np.einsum('aij->ija', X_aij)
X_ija.shape

(35, 24, 2)

In [20]:
X_ija[0,0,:]

array([476,   0])

# Evaluate a center of rotation from displacement data on a grid

Calculate the difference between the origin and the displacement at time $t$

$$
\Delta u_{tpa} = u_{tpa} - u_{0pa}
$$

Perpendicular vector to the movement of the point

$$
\perp u_{atp} = [\Delta u_{tp1}, -\Delta u_{tp0}]
$$

In [21]:
u1_tija.shape

(5, 35, 24, 2)

In [22]:
delta_u_ija = u1_tija[-1] - u1_tija[-2]
avg_a = np.average(delta_u_ija, axis=(0,1))
avg_a

array([-0.21408571, -2.11817857])

In [23]:
u_rot_ija = delta_u_ija - avg_a[np.newaxis,np.newaxis,:] 

(35, 24, 2)

In [34]:
rot_Xu_ija = X_ija + u_rot_ija * 100
rot_vect_u_nija = np.array([X_ija, rot_Xu_ija])
rot_vect_u_anij = np.einsum('nija->anij', rot_vect_u_nija)
rot_vect_u_anp = rot_vect_u_anij.reshape(2,2,-1)

In [43]:
rot_vect_u_anp

array([[[476.        , 476.        , 476.        , ...,   0.        ,
           0.        ,   0.        ],
        [519.10857143, 515.50857143, 511.80857143, ..., -24.89142857,
         -27.79142857, -30.69142857]],

       [[  0.        ,  14.        ,  28.        , ..., 294.        ,
         308.        , 322.        ],
        [ 65.71785714,  79.61785714,  94.01785714, ..., 239.11785714,
         252.41785714, 267.31785714]]])

In [44]:
perp_u_aij = np.array([u_rot_ija[...,1], -u_rot_ija[...,0]])
perp_u_ija = np.einsum('aij->ija', perp_u_aij)
perp_Xu_ija = X_ija + perp_u_ija * 100
perp_vect_u_nija = np.array([X_ija, perp_Xu_ija])
perp_vect_u_anij = np.einsum('nija->anij', perp_vect_u_nija)
perp_vect_u_anp = perp_vect_u_anij.reshape(2,2,-1)
perp_vect_u_anp

array([[[476.        , 476.        , 476.        , ...,   0.        ,
           0.        ,   0.        ],
        [541.71785714, 541.61785714, 542.01785714, ..., -54.88214286,
         -55.58214286, -54.68214286]],

       [[  0.        ,  14.        ,  28.        , ..., 294.        ,
         308.        , 322.        ],
        [-43.10857143, -25.50857143,  -7.80857143, ..., 318.89142857,
         335.79142857, 352.69142857]]])

In [48]:
import matplotlib.pylab as plt
fig, ax = plt.subplots(1,1)
ax.plot(*rot_vect_u_anp,color='blue');
ax.plot(*perp_vect_u_anp,color='green');
ax.axis('equal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
import scipy.stats as st
import matplotlib.pylab as plt
import numpy as np

In [61]:
x_data = np.linspace(-10,10,100)

In [65]:
fig, ax = plt.subplots(1,1)
ax.plot(x_data, st.norm.pdf(x_data, loc=0, scale=3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:

%pdoc st.norm

Class docstring:
    A normal continuous random variable.
    
    The location (``loc``) keyword specifies the mean.
    The scale (``scale``) keyword specifies the standard deviation.
    
    As an instance of the `rv_continuous` class, `norm` object inherits from it
    a collection of generic methods (see below for the full list),
    and completes them with details specific for this particular distribution.
    
    Methods
    -------
    rvs(loc=0, scale=1, size=1, random_state=None)
        Random variates.
    pdf(x, loc=0, scale=1)
        Probability density function.
    logpdf(x, loc=0, scale=1)
        Log of the probability density function.
    cdf(x, loc=0, scale=1)
        Cumulative distribution function.
    logcdf(x, loc=0, scale=1)
        Log of the cumulative distribution function.
    sf(x, loc=0, scale=1)
        Survival function  (also defined as ``1 - cdf``, but `sf` is sometimes more accurate).
    logsf(x, loc=0, scale=1)
        Log of the survival func

In [55]:
%pdoc st

Class docstring:
    .. _statsrefmanual:
    
    Statistical functions (:mod:`scipy.stats`)
    
    .. currentmodule:: scipy.stats
    
    This module contains a large number of probability distributions as
    well as a growing library of statistical functions.
    
    Each univariate distribution is an instance of a subclass of `rv_continuous`
    (`rv_discrete` for discrete distributions):
    
    .. autosummary::
       :toctree: generated/
    
       rv_continuous
       rv_discrete
       rv_histogram
    
    Continuous distributions
    
    .. autosummary::
       :toctree: generated/
    
       alpha             -- Alpha
       anglit            -- Anglit
       arcsine           -- Arcsine
       argus             -- Argus
       beta              -- Beta
       betaprime         -- Beta Prime
       bradford          -- Bradford
       burr              -- Burr (Type III)
       burr12            -- Burr (Type XII)
       cauchy            -- Cauchy
       chi       

In [266]:
np.sqrt(5**2+4**2)

6.4031242374328485

In [222]:
load_step_191_data_frame = pd.read_csv(
    'C:/Users/fahad/OneDrive/Documents/GOM Results/Load 191 KN.csv',sep=',',header=0,
     usecols = [0,2,4]) #np.array()
load_step_191_data_frame

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/fahad/OneDrive/Documents/GOM Results/Load 191 KN.csv'

In [ ]:
dx_data_frame = load_step_191_data_frame.iloc[::2]
dx_data_frame

In [4]:
dy_data_array = load_step_191_data_frame.iloc[1::2]
dy_data_array

,Element,Property,Actual
1,Point 2,dY,-5.978
3,Point 3,dY,-6.090
5,Point 4,dY,-6.204
7,Point 5,dY,-6.325
9,Point 6,dY,-6.493
...,...,...,...
1717,Point 860,dY,-9.345
1719,Point 861,dY,-9.449
1721,Point 862,dY,-9.534
1723,Point 863,dY,-5.866


In [5]:
load_step_191 = pd.read_csv('C:/Users/fahad/OneDrive/Documents/GOM Results/Load 191 KN.csv',sep = ',',header = 0,
                            usecols = [4]) #[0,2,4]
load_step_191

,Actual
0,1.651
1,-5.978
2,1.653
3,-6.090
4,1.651
...,...
1721,-9.534
1722,1.647
1723,-5.866
1724,-0.801


In [6]:
dx = load_step_191.iloc[::2]
dx_num = dx.to_numpy()
#dx_num

In [7]:
dy = load_step_191.iloc[1::2]
dy_num = dy.to_numpy()
#dy_num

In [8]:
d_array = np.array([dx_num, dy_num])
d_array_transpose = d_array.T
d_array_transpose

array([[[ 1.651, -5.978],
        [ 1.653, -6.09 ],
        [ 1.651, -6.204],
        ...,
        [-0.813, -9.534],
        [ 1.647, -5.866],
        [-0.801, -9.623]]])

In [9]:
import glob
import pandas as pd

glued_data = pd.DataFrame()
for file_name in glob.glob('C:/Users/fahad/OneDrive/Documents/GOM Results/*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)
glued_data

,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,0.000,NaN,NaN,0.000,NaN,NaN
1,Point 2,NaN,dY,0,0.000,NaN,NaN,0.000,NaN,NaN
2,Point 3,NaN,dX,0,0.000,NaN,NaN,0.000,NaN,NaN
3,Point 3,NaN,dY,0,0.000,NaN,NaN,0.000,NaN,NaN
4,Point 4,NaN,dX,0,0.000,NaN,NaN,0.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,-9.534,NaN,NaN,-9.534,NaN,NaN
1722,Point 863,NaN,dX,0,1.647,NaN,NaN,1.647,NaN,NaN
1723,Point 863,NaN,dY,0,-5.866,NaN,NaN,-5.866,NaN,NaN
1724,Point 864,NaN,dX,0,-0.801,NaN,NaN,-0.801,NaN,NaN


In [10]:
# import necessary libraries
import pandas as pd
import os
import glob
  
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "C:/Users/fahad/OneDrive/Documents/GOM Results/*.csv"))
  
  
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])
      
    # print the content
    print('Content:')
    display(df)
    print()

Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 0 KN.csv
File Name: Load 0 KN.csv
Content:


,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,0,NaN,NaN,0,NaN,NaN
1,Point 2,NaN,dY,0,0,NaN,NaN,0,NaN,NaN
2,Point 3,NaN,dX,0,0,NaN,NaN,0,NaN,NaN
3,Point 3,NaN,dY,0,0,NaN,NaN,0,NaN,NaN
4,Point 4,NaN,dX,0,0,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,0,NaN,NaN,0,NaN,NaN
1722,Point 863,NaN,dX,0,0,NaN,NaN,0,NaN,NaN
1723,Point 863,NaN,dY,0,0,NaN,NaN,0,NaN,NaN
1724,Point 864,NaN,dX,0,0,NaN,NaN,0,NaN,NaN



Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 050 KN.csv
File Name: Load 050 KN.csv
Content:


,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,0.743,NaN,NaN,0.743,NaN,NaN
1,Point 2,NaN,dY,0,-1.237,NaN,NaN,-1.237,NaN,NaN
2,Point 3,NaN,dX,0,0.742,NaN,NaN,0.742,NaN,NaN
3,Point 3,NaN,dY,0,-1.251,NaN,NaN,-1.251,NaN,NaN
4,Point 4,NaN,dX,0,0.745,NaN,NaN,0.745,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,-1.747,NaN,NaN,-1.747,NaN,NaN
1722,Point 863,NaN,dX,0,0.738,NaN,NaN,0.738,NaN,NaN
1723,Point 863,NaN,dY,0,-1.218,NaN,NaN,-1.218,NaN,NaN
1724,Point 864,NaN,dX,0,0.418,NaN,NaN,0.418,NaN,NaN



Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 103 KN.csv
File Name: Load 103 KN.csv
Content:


,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,1.229,NaN,NaN,1.229,NaN,NaN
1,Point 2,NaN,dY,0,-2.994,NaN,NaN,-2.994,NaN,NaN
2,Point 3,NaN,dX,0,1.223,NaN,NaN,1.223,NaN,NaN
3,Point 3,NaN,dY,0,-3.042,NaN,NaN,-3.042,NaN,NaN
4,Point 4,NaN,dX,0,1.224,NaN,NaN,1.224,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,-4.473,NaN,NaN,-4.473,NaN,NaN
1722,Point 863,NaN,dX,0,1.225,NaN,NaN,1.225,NaN,NaN
1723,Point 863,NaN,dY,0,-2.936,NaN,NaN,-2.936,NaN,NaN
1724,Point 864,NaN,dX,0,0.171,NaN,NaN,0.171,NaN,NaN



Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 150 KN.csv
File Name: Load 150 KN.csv
Content:


,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,1.434,NaN,NaN,1.434,NaN,NaN
1,Point 2,NaN,dY,0,-4.517,NaN,NaN,-4.517,NaN,NaN
2,Point 3,NaN,dX,0,1.436,NaN,NaN,1.436,NaN,NaN
3,Point 3,NaN,dY,0,-4.600,NaN,NaN,-4.600,NaN,NaN
4,Point 4,NaN,dX,0,1.438,NaN,NaN,1.438,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,-6.912,NaN,NaN,-6.912,NaN,NaN
1722,Point 863,NaN,dX,0,1.436,NaN,NaN,1.436,NaN,NaN
1723,Point 863,NaN,dY,0,-4.436,NaN,NaN,-4.436,NaN,NaN
1724,Point 864,NaN,dX,0,-0.280,NaN,NaN,-0.280,NaN,NaN



Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 191 KN.csv
File Name: Load 191 KN.csv
Content:


,Element,Datum,Property,Nominal,Actual,Tol -,Tol +,Dev,Check,Out
0,Point 2,NaN,dX,0,1.651,NaN,NaN,1.651,NaN,NaN
1,Point 2,NaN,dY,0,-5.978,NaN,NaN,-5.978,NaN,NaN
2,Point 3,NaN,dX,0,1.653,NaN,NaN,1.653,NaN,NaN
3,Point 3,NaN,dY,0,-6.090,NaN,NaN,-6.090,NaN,NaN
4,Point 4,NaN,dX,0,1.651,NaN,NaN,1.651,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1721,Point 862,NaN,dY,0,-9.534,NaN,NaN,-9.534,NaN,NaN
1722,Point 863,NaN,dX,0,1.647,NaN,NaN,1.647,NaN,NaN
1723,Point 863,NaN,dY,0,-5.866,NaN,NaN,-5.866,NaN,NaN
1724,Point 864,NaN,dX,0,-0.801,NaN,NaN,-0.801,NaN,NaN


In [11]:
# arr = np.zeros()
# a1 = np.array([[1, 2], [3, 4]])
# b1 = np.array([[5, 6], [7, 8]])
# np.vstack((a1,b1))

In [12]:
import pandas as pd
import os
import glob
  
  
'use glob to get all the csv files' 
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "C:/Users/fahad/OneDrive/Documents/GOM Results/*.csv"))

csv_files_num = len(csv_files)
displacement_3d_array = None
# looping over list of csv files
for f in csv_files:
    
    # reading csv file and converting into an array
    df = pd.read_csv(f)
    dx = df.iloc[::2]
    dx_num = dx.to_numpy()
    dy = df.iloc[1::2]
    dy_num = dy.to_numpy()
    d_array = np.array([dx_num, dy_num])
    d_array_transpose = d_array.T[4] 
    
    # printing location of file and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # printing values
    print('Displacements:')
    display(d_array_transpose)
    if displacement_3d_array is None:
        displacement_3d_array = d_array_transpose
    else:
        displacement_3d_array = np.vstack((displacement_3d_array, d_array_transpose))
    
    print()
print(displacement_3d_array)

Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 0 KN.csv
File Name: Load 0 KN.csv
Displacements:


array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=object)


Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 050 KN.csv
File Name: Load 050 KN.csv
Displacements:


array([[0.743, -1.237],
       [0.742, -1.251],
       [0.745, -1.266],
       ...,
       [0.417, -1.747],
       [0.738, -1.218],
       [0.418, -1.759]], dtype=object)


Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 103 KN.csv
File Name: Load 103 KN.csv
Displacements:


...


Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 150 KN.csv
File Name: Load 150 KN.csv
Displacements:


array([[1.434, -4.517],
       [1.436, -4.6],
       [1.438, -4.684],
       ...,
       [-0.275, -6.912],
       [1.436, -4.436],
       [-0.28, -6.958]], dtype=object)


Location: C:/Users/fahad/OneDrive/Documents/GOM Results\Load 191 KN.csv
File Name: Load 191 KN.csv
Displacements:


array([[1.651, -5.978],
       [1.653, -6.09],
       [1.651, -6.204],
       ...,
       [-0.813, -9.534],
       [1.647, -5.866],
       [-0.801, -9.623]], dtype=object)


[[0 0]
 [0 0]
 [0 0]
 ...
 [-0.813 -9.534]
 [1.647 -5.866]
 [-0.801 -9.623]]


In [13]:
displacement_3d_array

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [-0.813, -9.534],
       [1.647, -5.866],
       [-0.801, -9.623]], dtype=object)